## Máster en Ingeniería informática: Datos, Cloud y Gestión TI

# Análisis de información no estructurada

## Generador de letras de canciones

#### Realizado por: Arturo Pérez Sánchez y Jacinto Ruiz Díaz

El objetivo de este proyecto es el de implementar un sistema que permita generar de manera automática letras de canciones imitando el estilo de un artista en particular, realizando previamente una lectura de la letra de sus canciones.

El primer paso a realizar es la extracción de las letras de canciones para su posterior análisis. En nuestro caso, para obtener las letras de las canciones hemos optado por realizar <a href="https://es.wikipedia.org/wiki/Web_scraping">web scrapping</a> de la página <a href="https://www.azlyrics.com/">azlyrics</a>, la cual almacena letras de una amplia variedad de autores.

Para realizar la extracción de las letras utilizaremos la libreria <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">BeautifulSoup</a>, al ser una de las librerías de Python más completas e intuitivas para realizar este tipo de tareas.

Comenzaremos importándola:

In [1]:
import requests
from bs4 import BeautifulSoup

Una vez importada la librería definiremos algunos Hiperparámetros:
<ul>
    <li><strong>group:</strong> Nombre del grupo musical cuyo estilo deseamos imitar.</li>
    <li><strong>albums:</strong> Lista de álbumes del grupo que vamos a analizar (si se deja vacío se leerán todos).</li>
    <li><strong>wait_time:</strong> Tiempo en segundos que transcurrirá entre una petición y otra.</li>
</ul>

In [2]:
group = 'Lady Gaga'
albums = ['Red And Blue', 'The Fame', 'The Fame Monster', 'Born This Way', 'A Very Gaga Holiday', 'Artpop', 'Joanne', 'Chromatica']
wait_time = 5

Lo primero que tenemos que realizar es una conversión del nombre del grupo al nombre utilizado en la página (todo en minúsculas y sin espacios):

In [3]:
group = group.lower().replace(" ", "")

Ahora procedemos a estudiar cómo se estructuran las URLs del sitio web al que vamos a hacer scrapping.

Concretamente, estamos interesados en __dos__ vistas del sitio:

## Lista de canciones separadas por álbumes
La URL de esta página se estructura de la forma:
<h3><center>https://www.azlyrics.com/<font color="red">[initial]</font>/<font color="red">[group]</font>.html </center></h3>
Donde <i>[initial]</i> corresponde a la primera letra del grupo* y <i>[group]</i> corresponde al nombre del grupo. En el ejemplo del grupo Lady Gaga, se vería de la siguiente forma:

<img src="images/gaga_album_example.png" width=600>
<br>

*_Si el nombre del grupo comienza por un carácter especial la variable [initial] tomará el valor '19'_

## Letra de cada canción
En este caso, notaremos que la ruta es la misma pero cambiando [initial] por 'lyrics' y añadiendo el nombre de la canción justo después del nombre del grupo:

<h3><center> https://www.azlyrics.com/lyrics/<font color="red">[group]</font>/<font color="red">[song]</font>.html </center></h3>

El resultado para el ejemplo de la canción "Poker Face" de Lady Gaga es el siguiente:

<img src="images/gaga_lyrics2.png" width=600>

Con esto, ya podemos estructurar la ruta de álbumes a partir de los hiperparámetros que habíamos establecido:

In [4]:
#Inicializamos la variable initial como el primer caracter del grupo o 19 si no es una letra
initial = group[0] if group[0].isalpha() else '19'

#Prefijo del sitio web
prefix = 'https://www.azlyrics.com'
sufix = '.html'

#Concatenamos las variables para formar la URL completa
albumsUrl = prefix + '/' + initial + '/' + group

Con la URL de los álbumes ya creados, podemos utilizar las librerías anteriormente mencionadas para hacer la llamada a la página y obtener la lista de álbumes y los títulos de las canciones que lo componen:

In [5]:
# Las claves serán los álbumes y los valores un array con las canciones. Las canciones a su vez serán un array de tamaño 3:
# El primer valor es el titulo, el segundo el enlace y el tercero la letra de la canción
albumsDict = {}

# Hacemos la llamada para obtener la lista de álbumes
r = requests.get(albumsUrl + sufix)
soup = BeautifulSoup(r.text, 'lxml')

res = soup.findAll('div', class_=['album', 'listalbum-item'])

# Recorremos la lista de álbumes y para cada uno de ellos guardamos en el diccionario
# el título de cada canción y la ruta a la letra de dicha canción
currentAlbum = ''
for div in res:
    if(div.get('class')[0] == 'album'):
        # Este div corresponde a un álbum, por lo que añadimos una nueva entrada al diccionario
        currentAlbum = div.find('b').contents[0].replace('"', '')
        albumsDict[currentAlbum] = []
    else:
        # Este div corresponde a una canción por lo que la añadimos a los valores del álbum
        albumsDict[currentAlbum].append([div.contents[0].contents[0], 'https://www.azlyrics.com' + div.contents[0]['href'][2:]])

Ahora tenemos un diccionario con todos los álbumes, y para cada álbum tenemos todas las canciones que lo componen con sus respectivos enlaces a la letra. Por tanto, solo nos faltaría llamar a cada uno de estos enlaces para obtener la letra de cada canción; sin embargo, para reducir el número de llamadas al sitio web, primero realizaremos una limpieza de los álbumes para quedarnos solo con aquellos que habíamos indicado que nos interesaban:

In [6]:
# Limpieza de álbumes
# Del diccionario que hemos creado nos quedamos solo con los álbumes 
# que aparezcan en el array de álbumes que habíamos especificado inicialmente o con todos si no se ha especificado
if(albums):
    albumsDict = {a: albumsDict[a] for a in albums if a in albumsDict}

Ahora sí que tenemos todo listo para realizar la lectura de las canciones. Cabe destacar que entre cada llamada dejaremos un espacio de tiempo (indicado en la variable '_wait_time_') para evitar que el sitio web nos bloquee la dirección IP. Para conseguirlo haremos uso de la librería **time**.

In [7]:
import time

#Creamos un diccionario donde las claves serán las canciones y el valor la letra de dicha canción
lyricsDict = {}

print('Leyendo ' + str(len(albumsDict)) + ' álbum(es)')

#Recorremos la lista de álbumes y para cada álbum recorremos la lista de canciones
for album in albumsDict:
    print ('Reading album: ', album)
    for song in albumsDict[album]:
        print ('    Reading song: ', song[0])
        
        # Esperamos unos segundos antes de realizar la llamada
        time.sleep(wait_time)
        r = requests.get(song[1])
        soup = BeautifulSoup(r.text, 'lxml')
        
        # Como el div que contiene la letra no tiene ninguna clase ni identificador lo obtendremos a partir del div padre
        column = soup.find('div', class_=['col-xs-12 col-lg-8 text-center'])
        
        #El div con la letra de la canción siempre estará en 5º lugar, después de los divs del título y de las redes sociales
        raw_lyrics = column.findAll('div')[5].text
        lyricsDict[song[0]] = raw_lyrics

Leyendo 8 álbum(es)
Reading album:  Red And Blue
    Reading song:  Something Crazy
    Reading song:  Wish You Were Here
    Reading song:  No Floods
    Reading song:  Red And Blue
    Reading song:  Words
Reading album:  The Fame
    Reading song:  Just Dance
    Reading song:  LoveGame
    Reading song:  Paparazzi
    Reading song:  Beautiful, Dirty, Rich
    Reading song:  Eh, Eh (Nothing Else I Can Say)
    Reading song:  Poker Face
    Reading song:  The Fame
    Reading song:  Money Honey
    Reading song:  Again Again
    Reading song:  Boys Boys Boys
    Reading song:  Brown Eyes
    Reading song:  Summerboy
    Reading song:  I Like It Rough
    Reading song:  Retro Dance Freak
Reading album:  The Fame Monster
    Reading song:  Bad Romance
    Reading song:  Alejandro
    Reading song:  Monster
    Reading song:  Speechless
    Reading song:  Dance In The Dark
    Reading song:  Telephone
    Reading song:  So Happy I Could Die
    Reading song:  Teeth
Reading album:  Born 

Por último, tenemos que fusionar los arrays albumsDict y lyricsDict:

In [8]:
# Recorremos los álbumes
for album in albumsDict:
    # Recorremos las canciones de cada álbum
    for song in albumsDict[album]:
        #A cada canción le añadimos la letra correspondiente
        song.append(lyricsDict[song[0]])

Ahora ya tenemos todas las canciones tal y como queríamos, por lo que podemos guardarlas en un json para su posterior análisis:

In [9]:
import json 

# El json se guardará en la carpeta data
with open('data/' + group + "-lyrics.json", "w") as outfile: 
    json.dump(albumsDict, outfile)